In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import time
import pickle

In [2]:
###  Standard Cleaning Procedure  ###
def standard_df(df):
    df = df.iloc[1:]
    df = df.drop([0,1,3,6],axis=1)
    df.columns = ['Name','Entered Office','Exited Office','President']
    df = df.dropna(axis=0,how='all')
    df = df.fillna(method='ffill')
    df.Name = [name if 'Acting' not in name else np.nan for name in df.Name]
    df = df.dropna(axis=0,how='any')
    return df

In [3]:
###  URLs, Roles of Articles with White House Staffer Lists  ###
urls = ['https://en.wikipedia.org/wiki/List_of_Presidents_of_the_United_States',
        'https://en.wikipedia.org/wiki/List_of_Vice_Presidents_of_the_United_States',
        'https://en.wikipedia.org/wiki/List_of_Secretaries_of_State_of_the_United_States',
        'https://en.wikipedia.org/wiki/United_States_Secretary_of_Defense#List_of_Secretaries_of_Defense',
        'https://en.wikipedia.org/wiki/United_States_Secretary_of_the_Treasury#List_of_Secretaries_of_the_Treasury',
        'https://en.wikipedia.org/wiki/United_States_Secretary_of_Health_and_Human_Services#List_of_Secretaries_of_Health_and_Human_Services',
        'https://en.wikipedia.org/wiki/United_States_Attorney_General#List_of_Attorneys_General',
        'https://en.wikipedia.org/wiki/United_States_Secretary_of_Commerce#List_of_Secretaries_of_Commerce',
        'https://simple.wikipedia.org/wiki/United_States_Secretary_of_Labor',
        'https://en.wikipedia.org/wiki/United_States_Secretary_of_Housing_and_Urban_Development#List_of_Secretaries_of_Housing_and_Urban_Development',
        'https://en.wikipedia.org/wiki/United_States_Secretary_of_Transportation#Living_former_Secretaries_of_Transportation',
        'https://en.wikipedia.org/wiki/Administrator_of_the_Environmental_Protection_Agency',
        'https://en.wikipedia.org/wiki/Director_of_the_Federal_Bureau_of_Investigation',
        'https://en.wikipedia.org/wiki/United_States_Secretary_of_Homeland_Security#List_of_Secretaries_of_Homeland_Security',
        'https://en.wikipedia.org/wiki/National_Security_Advisor_(United_States)#List_of_National_Security_Advisors',
        'https://en.wikipedia.org/wiki/List_of_White_House_Chiefs_of_Staff']

roles = ['President',
        'Vice President',
        'Secretary of State',
        'Secretary of Defense',
        'Secretary of the Treasury',
        'Secretary of HHS',
        'Attorney General',
        'Secretary of Commerce',
        'Secretary of Labor',
        'Secretary of HUD',
        'Secretary of Transportation',
        'Director of FBI',
        'Secretary of Homeland Security',
        'Secretary of Homeland Security',
        'Chief of Staff']

In [4]:
###  Read URLs and attempt standard cleaning  ###

politician_data = []
for i, url in enumerate(urls):
    time.sleep(1)
    df = pd.read_html(url)[1]
    try:
        df = standard_df(df)
        df['Role'] = roles[i]
        politician_data.append(df)
    except:
        print(url)

https://en.wikipedia.org/wiki/List_of_Presidents_of_the_United_States
https://en.wikipedia.org/wiki/List_of_Vice_Presidents_of_the_United_States
https://en.wikipedia.org/wiki/List_of_Secretaries_of_State_of_the_United_States
https://en.wikipedia.org/wiki/United_States_Secretary_of_Defense#List_of_Secretaries_of_Defense
https://en.wikipedia.org/wiki/Administrator_of_the_Environmental_Protection_Agency
https://en.wikipedia.org/wiki/Director_of_the_Federal_Bureau_of_Investigation
https://en.wikipedia.org/wiki/National_Security_Advisor_(United_States)#List_of_National_Security_Advisors
https://en.wikipedia.org/wiki/List_of_White_House_Chiefs_of_Staff


In [5]:
###  Reading and Cleaning Dataframe for Chief of Staff  ###

df = pd.read_html('http://en.wikipedia.org/wiki/List_of_White_House_Chiefs_of_Staff')[0]
df = df.iloc[1:]
df = df.drop([0,1,2,5],axis=1)
df.columns = ['Name','Term','Party','President']
df = df[df.Name != 'Vacant']
df = df.fillna(method='ffill')
df['Role'] = 'Chief of Staff'
df.Name = df.Name.apply(lambda x :re.sub('[^a-zA-Z ]','',x))
df.President = df.President.apply(lambda x :re.sub('[^a-zA-Z ]','',x))
df['Entered Office'] = df['Term'].apply(lambda x: x.split('–')[0])
df['Exited Office'] = df['Term'].apply(lambda x: x.split('–')[1])
politician_data.append(df)
#df

In [6]:
###  Reading and Cleaning Dataframe for National Security Advisor  ###

df = pd.read_html('https://en.wikipedia.org/wiki/National_Security_Advisor_(United_States)#List_of_National_Security_Advisors')[1]
df = df.iloc[2:]
df = df.drop([0,1,5],axis=1)
df.columns = ['Name','Entered Office','Exited Office','President']
df = df.dropna(axis=0,how='all')
df = df.fillna(method='ffill')
df['Role'] = 'National Security Advisor'
df.Name = df.Name.apply(lambda x :re.sub('[^a-zA-Z ]','',x))
df['Exited Office'] = df['Exited Office'].apply(lambda x : x[:-3] if '[' in x else x)
df['Exited Office'] = df['Exited Office'].apply(lambda x : x[:-1] if '[' in x else x)
df['Entered Office'] = df['Entered Office'].apply(lambda x : x[:-3] if '[' in x else x)
df['Entered Office'] = df['Entered Office'].apply(lambda x : x[:-1] if '[' in x else x)
politician_data.append(df)
#df

In [7]:
###  Reading and Cleaning Dataframe for Head of FBI  ###

df = pd.read_html('https://en.wikipedia.org/wiki/Director_of_the_Federal_Bureau_of_Investigation')[2]
df = df.iloc[1:]
df = df.drop([0,1,4,5],axis=1)
df.columns = ['Name','Term','President']
df = df.fillna(method='ffill')
df.Name = [name if 'Acting' not in name else np.nan for name in df.Name]
df = df.dropna(axis=0,how='any')
df['Entered Office'] = df['Term'].apply(lambda x: x.split('–')[0])
df['Exited Office'] = df['Term'].apply(lambda x: x.split('–')[1])
df['Role'] = 'Head of FBI'
politician_data.append(df)
#df

In [8]:
###  Reading and Cleaning Dataframe for Head of EPA  ###

df = pd.read_html('https://en.wikipedia.org/wiki/Administrator_of_the_Environmental_Protection_Agency')[1]
df = df.iloc[1:]
df = df.drop([0],axis=1)
df.columns = ['Name','Term','President']
df = df.dropna(axis=0,how='all')
df = df.fillna(method='ffill')
df.Name = [name if 'Acting' not in name else np.nan for name in df.Name]
df = df.dropna(axis=0,how='any')
df['Entered Office'] = df['Term'].apply(lambda x: x.split('–')[0])
df['Exited Office'] = df['Term'].apply(lambda x: x.split('–')[1])
df['Role'] = 'Head of EPA'
politician_data.append(df)
#df

In [9]:
###  Reading and Cleaning Dataframe for Secretary of Defense  ###

df = pd.read_html('https://en.wikipedia.org/wiki/United_States_Secretary_of_Defense#List_of_Secretaries_of_Defense')[1]
df = df.iloc[1:]
df = df.drop([0,1,3,6,7],axis=1)
df.columns = ['Name','Entered Office','Exited Office','President']
df = df.dropna(axis=0,how='all')
df = df.fillna(method='ffill')
df.Name = [name if 'Acting' not in name else np.nan for name in df.Name]
df = df.dropna(axis=0,how='any')
df['Role'] = 'Secretary of Defense'
politician_data.append(df)
#df

In [10]:
###  Reading and Cleaning Dataframe for Secretary of State  ###

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_Secretaries_of_State_of_the_United_States')[2]
df = df.iloc[1:]
df = df.drop([0,1,3,6],axis=1)
df.columns = ['Name','Entered Office','Exited Office','President']
df = df.dropna(axis=0,how='all')
df = df.fillna(method='ffill')
df.Name = [name if 'Acting' not in name else np.nan for name in df.Name]
df = df.dropna(axis=0,how='any')
df.Name = df.Name.apply(lambda x : x[:-3] if '[' in x else x)
df['Exited Office'] = df['Exited Office'].apply(lambda x : x[:-3] if '[' in x else x)
df.President = df.President.apply(lambda x : x[:-3] if '[' in x else x)
df['Exited Office'] = df['Exited Office'].apply(lambda x : x[:-1] if '[' in x else x)
df['Exited Office'] = df['Exited Office'].apply(lambda x : x[:-2] if '[' in x else x)
df['Role'] = 'Secretary of State'
politician_data.append(df)
#df

In [11]:
###  Reading and Cleaning Dataframe for Vice President  ###

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_Vice_Presidents_of_the_United_States')[1]
df = df.iloc[2:]
df = df.drop([0,2,4,5,7],axis=1)
df.columns = ['Term','Name','Party','President']
df = df.dropna(axis=0,how='all')
df.President = df.President.fillna(method='ffill')
df = df.dropna(axis=0,how='any')
df.Name = df.Name.apply(lambda x: x[:20])
df.Name = df.Name.apply(lambda x :re.sub('[^a-zA-Z ]','',x))
df.Name = df.Name.apply(lambda x : x[:-5] if 'Born' in x else x)
df.Name = df.Name.apply(lambda x : x[:-3] if 'Bor' in x else x)
df.Name = df.Name.apply(lambda x : 'Al Gore' if 'Gore' in x else x)
df.President = df.President.apply(lambda x : x[:-3] if '[' in x else x)
df['Entered Office'] = df['Term'].apply(lambda x: x.split('–')[0])
df['Exited Office'] = df['Term'].apply(lambda x: x.split('–')[1])
df['Role'] = 'Vice President'
politician_data.append(df)
#df

In [12]:
###  Reading and Cleaning Dataframe for President  ###

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_Presidents_of_the_United_States')[1]
df = df.iloc[2:]
df = df.drop([0,2,4,5,7,8],axis=1)
df.columns = ['Term','Name','Party']
df = df.dropna(axis=0,how='all')
df.Party = df.Party.fillna(method='ffill')
df = df.dropna(axis=0,how='any')
df.Name = df.Name.apply(lambda x: x[:20])
df.Name = df.Name.apply(lambda x :re.sub('[^a-zA-Z ]','',x))
df.Name = df.Name.apply(lambda x : x[:-5] if 'Born' in x else x)
df.Name = df.Name.apply(lambda x : x[:-3] if 'Bor' in x else x)
df['Entered Office'] = df['Term'].apply(lambda x: x.split('–')[0])
df['Exited Office'] = df['Term'].apply(lambda x: x.split('–')[1])
df['Role'] = 'President'
df['President'] = df.Name
politician_data.append(df)
#df

In [69]:
df = politician_data[0]
for app_df in politician_data[1:]:
    df = df.append(app_df, sort=True)

df = df.reset_index()
df = df.drop('index',axis=1)
df = df.drop('Term',axis=1)
df.Party = df.Party.fillna('Not Available')
df['Entered Office'] = df['Entered Office'].apply(lambda x : x[:-3] if '[' in x else x)
df['President'] = df['President'].apply(lambda x : x.split('(')[0])
df['Party'] = df['Party'].apply(lambda x : x.split('(')[0])
df['Party'] = df['Party'].apply(lambda x : x.split('[')[0])
df['Exited Office'] = df['Exited Office'].apply(lambda x : x.split('(')[0])
df['Exited Office'] = df['Exited Office'].apply(lambda x : x.split('[')[0])
df['Entered Office'] = df['Entered Office'].apply(lambda x : x.split('[')[0])
df = df.drop(index=[18,81,90], axis=0) #ambda x : x.split()[0])
df = df.reset_index()
df = df.drop('index',axis=1)
df

,Entered Office,Exited Office,Name,Party,President,Role
0,"September 11, 1789","January 31, 1795",Alexander Hamilton,Not Available,George Washington,Secretary of the Treasury
1,"February 3, 1795","December 31, 1800","Oliver Wolcott, Jr.",Not Available,George Washington,Secretary of the Treasury
2,"January 1, 1801","May 13, 1801",Samuel Dexter,Not Available,George Washington,Secretary of the Treasury
3,"May 14, 1801","February 8, 1814",Albert Gallatin,Not Available,George Washington,Secretary of the Treasury
4,"February 9, 1814","October 5, 1814",George W. Campbell,Not Available,George Washington,Secretary of the Treasury
5,"October 6, 1814","October 21, 1816",Alexander J. Dallas,Not Available,George Washington,Secretary of the Treasury
6,"October 22, 1816","March 6, 1825",William H. Crawford,Not Available,George Washington,Secretary of the Treasury
7,"March 7, 1825","March 5, 1829",Richard Rush,Not Available,John Quincy Adams,Secretary of the Treasury
8,"March 6, 1829","June 20, 1831",Samuel D. Ingham,Not Available,Andrew Jackson,Secretary of the Treasury
9,"August 8, 1831","May 28, 1833",Louis McLane,Not Available,Andrew Jackson,Secretary of the Treasury


In [70]:
ent_dates = []
for item in df['Entered Office']:
    try:
        ent_dates.append(datetime.strptime(item, '%B %d, %Y'))
    except:
        try:
            l = item.split()
            w = ' '.join(l)
            ent_dates.append(datetime.strptime(w, '%B %d, %Y'))
        except:
            try:
                ent_dates.append(datetime.strptime(item, '%d %B %Y'))
            except:
                ent_dates.append(item.lower())
df['Entered Office'] = ent_dates

ex_dates = []
for item in df['Exited Office']:
    try:
        ex_dates.append(datetime.strptime(item, '%B %d, %Y'))
    except:
        try:
            l = item.split()
            w = ' '.join(l)
            ex_dates.append(datetime.strptime(w, '%B %d, %Y'))
        except:
            try:
                ex_dates.append(datetime.strptime(item, '%d %B %Y'))
            except:
                ex_dates.append(item.lower().strip(' '))
df['Exited Office'] = ex_dates

In [71]:
df = df.sort_values('Entered Office')
df = df.reset_index()
df = df.drop('index',axis=1)
df = df.iloc[100:]
df = df.reset_index()
df = df.drop('index',axis=1)
df.Name = df.Name.apply(lambda x: x.lower())
df.Name = df.Name.apply(lambda x: re.sub('[^a-z ]', '', x))
df['collectionname'] = df.Name.apply(lambda x: re.sub('[ ]','',x))

In [85]:
df.head()

,Entered Office,Exited Office,Name,Party,President,Role,collectionname
0,1861-03-04,1865-04-15 00:00:00,abraham lincoln,Republican,Abraham Lincoln,President,abrahamlincoln
1,1861-03-04,1865-03-04 00:00:00,hannibal hamlin,Republican,Abraham Lincoln[k],Vice President,hannibalhamlin
2,1861-03-05,1864-11-24 00:00:00,edward bates,Not Available,Abraham Lincoln,Attorney General,edwardbates
3,1861-03-05,1865-04-15 00:00:00,william h seward,Not Available,Abraham Lincoln,Secretary of State,williamhseward
4,1861-03-07,1864-06-30 00:00:00,salmon p chase,Not Available,Abraham Lincoln,Secretary of the Treasury,salmonpchase


In [73]:
df.to_pickle('pol_df.pkl')

In [74]:
df1 = pd.read_pickle('pol_df.pkl')
#df1

In [75]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
politician_db = client['politician_db_test']

In [87]:
from nytimesarticle import articleAPI
api = articleAPI('55855057701943e2bebecb6f113bce71')
df1 = df1.iloc[0:1]
for politician in df1.Name:
    df_entry = df1[df1.Name == politician]
    search = api.search(q = politician,
                       begin_date=18610304,
                       end_date=18650415)
    politician_col = politician_db[df_entry['collectionname'][df_entry.index[0]]]
    print('success for ', politician)
    print(search['response']['docs'])
    politician_col.insert_many(search['response']['docs'])
    

success for  abraham lincoln
[{'web_url': 'https://www.nytimes.com/1861/12/24/news/sons-oe-new-england-anniversary-dinner-astor-house-superb-civil-military.html', 'snippet': '', 'print_page': '8', 'blog': {}, 'source': 'The New York Times', 'multimedia': [], 'headline': {'main': 'THE SONS OE NEW-ENGLAND.; Anniversary Dinner at the Astor House. A Superb Civil and Military Entertainment.Addresses by Wm. M. Evarts, Esq., Rev. Drs.Adams and Storrs, Hon. Robt. J. Walker,Hon. Charles Anderson, Gov.Curtin, Hon. H. J. Raymond, and Others.Anthem by Wm. Ross Wallace, and a Poem by John G. Saxe.Letters from president Lincoln, Secretary Seward, Lord Lyons, Governors Morgan, Andrew, and Others. PILGRIM.', 'kicker': None, 'content_kicker': None, 'print_headline': None, 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'persons', 'value': 'SAXE, JOHN GODFREY', 'rank': 0, 'major': None}, {'name': 'organizations', 'value': 'NEW ENGLAND SOCIETY', 'rank': 0, 'major': None}, {'name': 'subject

In [88]:
len(search['response']['docs'])

10

In [84]:
politician_col.find_one()

{'_id': '5bf244a33a125f5075bfaf31',
 'web_url': 'https://www.nytimes.com/aponline/2018/11/19/us/ap-history.html',
 'snippet': 'Today in History',
 'blog': {},
 'source': 'AP',
 'multimedia': [],
 'headline': {'main': 'Today in History',
  'kicker': None,
  'content_kicker': None,
  'print_headline': 'Today in History',
  'name': None,
  'seo': None,
  'sub': None},
 'keywords': [],
 'pub_date': '2018-11-19T05:05:36+0000',
 'document_type': 'article',
 'news_desk': 'None',
 'byline': {'original': 'By THE ASSOCIATED PRESS',
  'person': [],
  'organization': 'THE ASSOCIATED PRESS'},
 'type_of_material': 'News',
 'word_count': 842,
 'score': 64.04402,
 'uri': 'nyt://article/6a8ec232-1d07-53f0-a0ed-7f7be06a9bd1'}